In [30]:
import pandas as pd
from requests import get
from transformers import AutoModel, AutoTokenizer
from razdel import tokenize
from razdel import tokenize
import re
import nltk
import pymorphy2
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from artm import BatchVectorizer

train_data_link = 'https://huggingface.co/datasets/mlsum/resolve/refs%2Fconvert%2Fparquet/ru/train/0000.parquet?download=true'
test_data_link = 'https://huggingface.co/datasets/mlsum/resolve/refs%2Fconvert%2Fparquet/ru/test/0000.parquet?download=true'

rus_stop_words = stopwords.words("russian")
symbols = list(punctuation) + ['\t', '\n', '\r', '№', '#', '@', '^', '%', '*', '»', '«', '×', 'ń', '√', 'а́', '—']
emoj = re.compile('['
                  u'\U0001F600-\U0001F64F'
                  u'\U0001F300-\U0001F5FF'
                  u'\U0001F680-\U0001F6FF'
                  u'\U0001F1E0-\U0001F1FF'
                  u'\U00002500-\U00002BEF'
                  u'\U00002702-\U000027B0'
                  u'\U000024C2-\U0001F251'
                  u'\U0001f926-\U0001f937'
                  u'\U00010000-\U0010ffff'
                  u'\u2640-\u2642'
                  u'\u2600-\u2B55'
                  u'\u200d'
                  u'\u23cf'
                  u'\u23e9'
                  u'\u231a'
                  u'\ufe0f'
                  u'\u3030'
                  ']', re.UNICODE)

morph = pymorphy2.MorphAnalyzer(lang='ru')

tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
vectorizer = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

with open('./less_semantic_words.txt', 'r', encoding='utf-8') as f:
    stop_words = pd.Series(f.read().split('\n')).unique()

ModuleNotFoundError: No module named 'artm'

# init data

In [2]:
with open('train_data.parquet', 'wb') as train_df:
    train_df.write(get(train_data_link).content)
    
with open('test_data.parquet', 'wb') as test_df:
    test_df.write(get(test_data_link).content)

In [5]:
train_data = pd.read_parquet('train_data.parquet', engine='pyarrow')
train_data.head(3)

,text,summary,topic,url,title,date
0,Сладострастник в течение трех лет преследовал ...,Старший преподаватель института коммунального ...,incident,https://www.mk.ru/incident/article/2010/01/05/...,Педофил преследовал подростка три года,06/01/2010
1,Буквально за час до боя курантов в подземном п...,Манежная площадь Москвы стала местом последнег...,incident,https://www.mk.ru/incident/article/2010/01/05/...,Таджики устроили резню на Манежной площади под...,06/01/2010
2,"Там они покатались на лыжах и снегоходах, пооб...",Президент РФ Дмитрий Медведев с семьей проводи...,politics,https://www.mk.ru/politics/article/2010/01/05/...,"""""""За все платит президент""""""",06/01/2010


In [23]:
def filter_fun(text):
    is_not_symbol = True
    for i in text:
        if i in symbols:
            is_not_symbol = False
    is_not_num = not bool(re.match('\d+', text))
    is_not_stop_word = not text in rus_stop_words
    is_not_stop_word_2 = not text in stop_words 
    return is_not_num and is_not_symbol and is_not_stop_word and is_not_stop_word_2


In [24]:
def textNormalizer(text: str):
    text = text.lower()
    text = list(map(lambda a: a.text, tokenize(text)))
    text = list(filter(filter_fun, text))
    text = [re.sub(emoj, ' ', msg) for msg in text]
    text = list(map(lambda i: morph.parse(i)[0].normal_form, text))
    return ' '.join(text)


In [25]:
# text_normalized = train_data['text'].apply(textNormalizer)
docs = [textNormalizer(t) for t in tqdm(train_data['text'])]

100%|██████████| 25556/25556 [30:26<00:00, 14.00it/s]


In [26]:
train_data['text_processed'] = docs

In [27]:
train_data.to_csv('./train_data_cleared.csv', index=False)

# work

In [28]:
train_data = pd.read_csv('./train_data_cleared.csv')